In [2]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import motmetrics as mm

def load_yolo_model(model_path):
    """
    Load the YOLOv8 model.
    """
    model = YOLO(model_path)
    return model

def initialize_deepsort(max_age=30, n_init=3, nms_max_overlap=1.0, max_cosine_distance=0.3, nn_budget=None, override_track_class=None):
    """
    Initialize the DeepSORT tracker with adjustable parameters.
    """
    tracker = DeepSort(max_age=max_age,
                       n_init=n_init,
                       nms_max_overlap=nms_max_overlap,
                       max_cosine_distance=max_cosine_distance,
                       nn_budget=nn_budget,
                       override_track_class=override_track_class)
    return tracker

def load_ground_truth(gt_path):
    """
    Load ground truth data from a file.
    """
    gt_data = []
    with open(gt_path, 'r') as f:
        for line in f:
            parts = line.strip().split(',')
            frame_id, track_id, x, y, w, h, _, _, _ = map(int, parts)
            gt_data.append((frame_id, track_id, x, y, w, h))
    return gt_data

def process_sequence(seq_path, yolo_model, deepsort_tracker, target_class_ids=None, confidence_threshold=0.3):
    """
    Process a single sequence folder and return tracking results.
    """
    img_dir = os.path.join(seq_path, "img1")
    img_files = sorted([f for f in os.listdir(img_dir) if f.endswith('.jpg')])

    # List to store tracking results
    tracking_results = []

    for frame_idx, img_file in enumerate(img_files):
        img_path = os.path.join(img_dir, img_file)
        frame = cv2.imread(img_path)
        if frame is None:
            print(f"Warning: Unable to load frame {img_path}")
            continue

        # Run YOLO detection on the frame
        results = yolo_model(frame)[0]

        detections = []
        for result in results:
            boxes = result.boxes.xywh.cpu().numpy()
            confs = result.boxes.conf.cpu().numpy()
            class_ids = result.boxes.cls.cpu().numpy()

            for box, conf, cls in zip(boxes, confs, class_ids):
                if conf > confidence_threshold:
                    if target_class_ids is None or int(cls) in target_class_ids:
                        x_center, y_center, w, h = box
                        x1 = int(x_center - (w / 2))
                        y1 = int(y_center - (h / 2))
                        x2 = int(x_center + (w / 2))
                        y2 = int(y_center + (h / 2))
                        detections.append([[x1, y1, x2 - x1, y2 - y1], conf, cls])

        # Update DeepSORT tracker
        tracks = deepsort_tracker.update_tracks(detections, frame=frame)

        # Save tracking results
        for track in tracks:
            if not track.is_confirmed():
                continue
            track_id = track.track_id
            bbox = track.to_ltrb()
            tracking_results.append((frame_idx + 1, track_id, bbox[0], bbox[1], bbox[2] - bbox[0], bbox[3] - bbox[1]))

        if frame_idx % 50 == 0:
            print(f"Processed frame {frame_idx}/{len(img_files)} in {seq_path}")

    return tracking_results

def evaluate_tracking_results(tracking_results, gt_data):
    """
    Evaluate tracking results using motmetrics.
    """
    acc = mm.MOTAccumulator(auto_id=True)

    for frame_id in range(1, max([x[0] for x in gt_data]) + 1):
        gt_objects = [x for x in gt_data if x[0] == frame_id]
        tr_objects = [x for x in tracking_results if x[0] == frame_id]

        gt_bboxes = [[x[2], x[3], x[4], x[5]] for x in gt_objects]
        gt_ids = [x[1] for x in gt_objects]

        tr_bboxes = [[x[2], x[3], x[4], x[5]] for x in tr_objects]
        tr_ids = [x[1] for x in tr_objects]

        # Compute IoU matrix manually
        def compute_iou(box1, box2):
            x1 = max(box1[0], box2[0])
            y1 = max(box1[1], box2[1])
            x2 = min(box1[0] + box1[2], box2[0] + box2[2])
            y2 = min(box1[1] + box1[3], box2[1] + box2[3])
            inter_area = max(0, x2 - x1) * max(0, y2 - y1)
            box1_area = box1[2] * box1[3]
            box2_area = box2[2] * box2[3]
            union_area = box1_area + box2_area - inter_area
            iou = inter_area / union_area if union_area > 0 else 0
            return iou

        distances = []
        for gt_box in gt_bboxes:
            row = []
            for tr_box in tr_bboxes:
                iou = compute_iou(gt_box, tr_box)
                row.append(1 - iou)  # Distance is 1 - IoU
            distances.append(row)

        acc.update(gt_ids, tr_ids, distances)

    # Compute metrics
    mh = mm.metrics.create()
    summary = mh.compute(acc, metrics=['idf1', 'precision', 'mota'], name='acc')
    return summary

def process_all_sequences(val_folder, yolo_model, deepsort_tracker, target_class_ids=None, confidence_threshold=0.3):
    """
    Process all sequence folders in the validation directory and compute average metrics.
    """
    sequence_folders = [os.path.join(val_folder, seq) for seq in os.listdir(val_folder) if os.path.isdir(os.path.join(val_folder, seq))]
    all_metrics = []

    for seq_folder in sequence_folders:
        print(f"Processing sequence: {seq_folder}")
        gt_path = os.path.join(seq_folder, "gt", "gt.txt")

        if not os.path.exists(gt_path):
            print(f"Warning: Ground truth file not found for {seq_folder}")
            continue

        # Load ground truth
        gt_data = load_ground_truth(gt_path)

        # Process sequence
        tracking_results = process_sequence(seq_folder, yolo_model, deepsort_tracker, target_class_ids, confidence_threshold)

        # Evaluate tracking results
        summary = evaluate_tracking_results(tracking_results, gt_data)
        all_metrics.append(summary)

        print(f"Metrics for {seq_folder}:")
        print(summary)

    # Compute average metrics
    avg_idf1 = np.mean([metrics['idf1'] for metrics in all_metrics])
    avg_mota = np.mean([metrics['mota'] for metrics in all_metrics])
    avg_precision = np.mean([metrics['precision'] for metrics in all_metrics])

    print("\nAverage Metrics:")
    print(f"IDF1: {avg_idf1:.4f}")
    print(f"MOTA: {avg_mota:.4f}")
    print(f"Precision: {avg_precision:.4f}")

# Paths
yolo_model_path = '/home/ubuntu/project_dl/src/runs/detect/train/weights/best.pt'
val_folder = '/home/ubuntu/project_dl/dataset/onlyfootball/dataset/val/'

# Load YOLO model and initialize DeepSORT
yolo_model = load_yolo_model(yolo_model_path)
deepsort_tracker = initialize_deepsort(max_age=30, n_init=3, max_cosine_distance=0.2)

# Process all sequences and compute average metrics
process_all_sequences(val_folder, yolo_model, deepsort_tracker, target_class_ids=[0])

Processing sequence: /home/ubuntu/project_dl/dataset/onlyfootball/dataset/val/v_dw7LOz17Omg_c067



0: 736x1280 17 players, 5.1ms
Speed: 4.1ms preprocess, 5.1ms inference, 1.8ms postprocess per image at shape (1, 3, 736, 1280)
Processed frame 0/450 in /home/ubuntu/project_dl/dataset/onlyfootball/dataset/val/v_dw7LOz17Omg_c067

0: 736x1280 17 players, 3.7ms
Speed: 2.4ms preprocess, 3.7ms inference, 0.9ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 17 players, 3.6ms
Speed: 2.2ms preprocess, 3.6ms inference, 0.8ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 17 players, 3.5ms
Speed: 2.1ms preprocess, 3.5ms inference, 0.7ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 17 players, 3.6ms
Speed: 2.1ms preprocess, 3.6ms inference, 0.7ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 17 players, 3.5ms
Speed: 2.2ms preprocess, 3.5ms inference, 0.8ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 17 players, 3.6ms
Speed: 2.2ms preprocess, 3.6ms inference, 0.7ms postprocess per image at shape (1, 3, 736, 1280)

0: 